In [176]:
import math
import csv
from dataclasses import dataclass
from typing import List, Tuple

Постановка. Задача электроразведки. Однородное полупространство. Источник поля – заземлённая электрическая линия с постоянным
значением силы тока $$I = 1 A $$ c координатами электродов $$A(0,0,0), B(100,0,0).$$

Измеряется разность потенциалов в приёмных линиях с координатами $$M_1(200,0,0), N_1(300,0,0);$$ 
$$M_2(500,0,0), N_2(600,0,0);$$
$$M_3(1000,0,0), N_3(1100,0,0).$$

Неизвестным является значение удельной
электрической проводимости. В качестве истинного значения выбрать
$σ = 0.1 См/м.$ В качестве начального значения выбрать $σ^0 = 0.01 См/м.$


<b>Геометрия задачи (координаты на оси x)</b>

In [177]:
A_x = 0.0
B_x = 100.0
M = [200.0, 500.0, 1000.0]
N = [300.0, 600.0, 1100.0]

I = 1.0

Разность потенциалов в линии MN
$$ V_{AB}^{MN} = \frac{I}{2\pi\sigma} \left( \left( \frac{1}{r_B^M} - \frac{1}{r_A^M} \right) - \left( \frac{1}{r_B^N} - \frac{1}{r_A^N} \right) \right)$$

Функция для вычисления геометрических коэффициентов (без $\sigma$) $$ K = \frac{I}{2\pi} \left( \left( \frac{1}{r_B^M} - \frac{1}{r_A^M} \right) - \left( \frac{1}{r_B^N} - \frac{1}{r_A^N} \right) \right)$$
Попарно по всем точкам $M$ и $N$

In [178]:
def calc_K(Ax: float, Bx: float, M: List[float], N: List[float], I: float) -> List[float]:
    K = []
    for Mi, Ni in zip(M, N):
        term = (1.0 / abs(Mi - Bx) - 1.0 / abs(Mi - Ax)) - (1.0 / abs(Ni - Bx) - 1.0 / abs(Ni - Ax))
        K.append(I / (2.0 * math.pi) * term)
    return K

Теперь можно вычислить $V_i(σ)$ по формуле $$ V_i(σ) = K_i / σ $$

In [179]:
def V_of_sigma(sigma: float, K: List[float]) -> List[float]:
    return [k / sigma for k in K]

И производные от них $$ dV_i/dσ = -K_i / σ^2 $$

In [180]:
def dV_dsigma(sigma: float, K: List[float]) -> List[float]:
    return [-k / (sigma**2) for k in K]

Функционал обратной задачи $$ \Phi(\sigma) = \sum_{i=1}^{3} \left( w_i \left( V_i(\sigma) - \bar{V}_i \right) \right)^2 \to \min_{\sigma} $$

In [181]:
def phi(sigma: float, K: List[float], Vbar: List[float], w: List[float]) -> float:
    Vi = V_of_sigma(sigma, K)
    return sum((wi * (v - vb))**2 for wi, v, vb in zip(w, Vi, Vbar))

Итерационный процесс метода Гаусса-Ньютона
$$\frac{\partial V_i(\sigma)}{\partial \sigma} = -\frac{I}{2\pi\sigma^2} \left( \left( \frac{1}{r_B^{M_i}} - \frac{1}{r_A^{M_i}} \right) - \left( \frac{1}{r_B^{N_i}} - \frac{1}{r_A^{N_i}} \right) \right)$$
$$ a_{11} = \sum_{i=1}^{3} \left( w_i \frac{\partial V_i(\sigma)}{\partial \sigma} \right)^2 \bigg|_{\sigma = \sigma^0} $$
$$ b_1 = -\sum_{i=1}^{3} \left( w_i^2 \frac{\partial V_i(\sigma)}{\partial \sigma} (V_i(\sigma) - \bar{V}_i(\sigma)) \right) \bigg|_{\sigma = \sigma^0} $$
$$\Delta\sigma^{01} = b_1 / a_{11} $$
$$ \sigma^1 = \sigma^0 + \Delta\sigma^{01}$$

In [182]:
@dataclass
class GNResult:
    iter: int
    sigma: float
    phi: float
    delta_sigma: float

In [183]:
def gauss_newton_1param(
    sigma0: float,
    K: List[float],
    Vbar: List[float],
    w: List[float],
    tol: float = 1e-9,
    itmax: int = 50
) -> Tuple[float, List[GNResult]]:
    hist: List[GNResult] = []
    sigma = sigma0
    hist.append(GNResult(0, sigma, phi(sigma, K, Vbar, w), 0.0))
    for it in range(1, itmax + 1):
        dVi = dV_dsigma(sigma, K)
        a11 = sum((wi * d)**2 for wi, d in zip(w, dVi))
        b1 = -sum((wi**2) * d * (v - vb)
                  for wi, d, v, vb in zip(w, dVi, V_of_sigma(sigma, K), Vbar))
        delta = b1 / a11
        sigma = sigma + delta
        hist.append(GNResult(it, sigma, phi(sigma, K, Vbar, w), delta))
        if abs(delta) < tol:
            break
    return sigma, hist

Вычислим коэффициенты $K_i$

In [184]:
K = calc_K(A_x, B_x, M, N, I)
K

[0.0005305164769729845, 2.6525823848649238e-05, 3.2152513755938386e-06]

Вычислим "синтетические" данные $$\sigma_{true}=0.1$$ $$ \bar{V}=K/\sigma_{true}$$

In [185]:
sigma_true = 0.1
Vbar = V_of_sigma(sigma_true, K)
Vbar

[0.005305164769729845, 0.00026525823848649237, 3.2152513755938386e-05]

Весовые коэффициенты $w_i$ вычислим по формуле $$ w_i = 1 / \bar{V}_i $$

In [186]:
w = [1.0 / vb for vb in Vbar]
w

[188.49555921538757, 3769.91118430775, 31101.767270539018]

Запустим итерационный процесс с начальным значением $\sigma_0 = 0.01$

In [187]:
sigma0 = 0.01
sigma_est, history = gauss_newton_1param(sigma0, K, Vbar, w)

In [188]:
# print(["iter", "sigma", "phi", "delta_sigma"])
print(["iter", "sigma", "phi"])
for r in history:
    # print([r.iter, f"{r.sigma:.12e}", f"{r.phi:.12e}", f"{r.delta_sigma:.12e}"])
    print([r.iter, f"{r.sigma:.12e}", f"{r.phi:.12e}"])

['iter', 'sigma', 'phi']
[0, '1.000000000000e-02', '2.430000000000e+02']
[1, '1.900000000000e-02', '5.452354570637e+01']
[2, '3.439000000000e-02', '1.091935482371e+01']
[3, '5.695327900000e-02', '1.713815000531e+00']
[4, '8.146979811148e-02', '1.551987705620e-01']
[5, '9.656631617971e-02', '3.793067142692e-03']
[6, '9.988209815422e-02', '4.180104601577e-06']
[7, '9.999986099155e-02', '5.797021065077e-12']
[8, '9.999999999981e-02', '1.120326533826e-23']
[9, '1.000000000000e-01', '0.000000000000e+00']


In [189]:
print(f"Оценка σ = {sigma_est:.12f} S/m")

Оценка σ = 0.100000000000 S/m
